- bitflyerのAPI設定
- ティッカーとか板情報の取得

In [5]:
import configparser
import time

import ccxt
from tqdm import tqdm

In [2]:
# API

conf = configparser.ConfigParser()
conf.read('../../../data/bot/settings.ini')

api_key = conf['bitflyer']['api_key']
api_secret = conf['bitflyer']['api_secret']

# CCXT
bitflyer = ccxt.bitflyer()

bitflyer.apiKey = api_key
bitflyer.secret = api_secret

In [8]:
# 5分で300回以上リクエストを送るとエラー
# https://lightning.bitflyer.com/docs?lang=ja#http-private-api

In [5]:
# 口座残高の確認

# bitflyer.private_get_getbalance()

In [3]:
params = {
    'product_code': 'BTC_JPY'
}

In [9]:
# ティッカー

## depthは注文総数

bitflyer.public_get_getticker(params=params)

{'product_code': 'BTC_JPY',
 'state': 'RUNNING',
 'timestamp': '2021-03-17T06:07:29.63',
 'tick_id': 38123732,
 'best_bid': 6113801.0,
 'best_ask': 6115942.0,
 'best_bid_size': 0.08793756,
 'best_ask_size': 0.1226,
 'total_bid_depth': 768.27362435,
 'total_ask_depth': 644.30390324,
 'market_bid_size': 0.0,
 'market_ask_size': 0.0,
 'ltp': 6113801.0,
 'volume': 21983.9506305,
 'volume_by_product': 7222.94703904}

In [11]:
# 板

# bitflyer.public_get_getboard(params)

In [13]:
# 約定データの蓄積


In [14]:
# 板情報の蓄積
